In [6]:
 # for maximize the cell or screen  in here  
from IPython.display import HTML

HTML("<style>.container { width:100% !important; }</style>")

In [43]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException,ElementNotInteractableException,TimeoutException
from selenium import webdriver
import time
import pandas as pd

from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options


In [46]:
def get_jobs(keyword, num_jobs, verbose):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.maximize_window()

    wait=WebDriverWait(driver,10)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    values=[]

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
        time.sleep(4)
        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CSS_SELECTOR,"#SignInButton").click()
            #wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#SignInButton"))).click()
        except ElementNotInteractableException:
            pass
        time.sleep(1)
        
        try:
            driver.find_element(By.CSS_SELECTOR,"button.CloseButton").click() #clicking to the X. 
        except NoSuchElementException:
            pass
        
        #Going through each job in this page
        #job_buttons = driver.find_elements(By.XPATH,'//*[@id="left-column"]/div/ul[@aria-label="Jobs List"]/li') #jl for Job Listing. These are the buttons we're going to click.
        job_buttons=driver.find_elements(By.XPATH,'*//a[@class="JobCard_trackingLink__GrRYn"]')
        print(len(set(job_buttons)))
        for job_button in job_buttons: 
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            
            values=[]
            values.append(job_button.get_attribute('aria-labelledby'))
            
            ActionChains(driver).move_to_element(job_button).click().perform()  #You might 
            time.sleep(5)
            
            try:
                company_name = driver.find_element(By.ID,values[0].split(" ")[3]).text
                location = driver.find_element(By.ID,values[0].split(" ")[2]).text
                job_title = driver.find_element(By.ID,values[0].split(" ")[0]).text
                salary =driver.find_element(By.ID,values[0].split(" ")[1]).text
            except:
                time.sleep(5)
            values.clear()
            
            jobs.append({"Company Name" : company_name,
                        "Location" : location,
                         "Job_Title" : job_title,
                         "Salary Estimate" : salary,
                         #"Job_Description" : job_description,
                         #"Skill" : skill
                        })
            if len(jobs)>num_jobs: break

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH,'//*[@id="left-column"]/div[2]/div/button').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [47]:
df=get_jobs('Data Scientist',10 , False)
df

30
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Company Name,Location,Job_Title,Salary Estimate
0,Salesforce\n4.1,"San Francisco, CA",Data Scientist,$126K - $197K (Employer est.)
1,Scale AI\n3.7,"San Francisco, CA","Data Scientist, Growth/Marketing",$138K - $219K (Glassdoor est.)
2,Pinterest\n3.9,"San Francisco, CA",Data Scientist II,$119K - $209K (Employer est.)
3,Visa\n4.1,"Foster City, CA",Data Scientist,$126K - $177K (Employer est.)
4,"Intelliswift Software, Inc.\n3.8","San Francisco, CA",Product Analytics Data Scientist,$104.23 Per Hour (Employer est.)
5,Meta\n3.9,"Menlo Park, CA","Data Scientist, Business Marketing Group",$109K - $171K (Employer est.)
6,University of California San Francisco\n4.1,"San Francisco, CA",Statistician and Data Scientist,$91K - $156K (Glassdoor est.)
7,Quanata\n4.6,"San Francisco, CA",Associate Actuarial Data Scientist [Remote],$101K - $163K (Glassdoor est.)
8,Paradyme Management\n3.5,"Redwood City, CA",Data Scientist,$114K - $167K (Glassdoor est.)
9,BetterHelp\n3.9,"Mountain View, CA",Data Scientist,$110K - $150K (Employer est.)
